In [ ]:
# Install packages
install.packages("haven")
install.packages("ggplot2")
install.packages("survey")
# Load the packages
library(conflicted)
library(haven)
library(ggplot2)
library(dplyr)
library(scales)  # Load the scales library for formatting
library(gridExtra)
library(forcats) 
library(readr)
library(gridExtra)
library(caret)
library(stringr)
library(purrr) # For map_df()
# Use survey package 
library(survey)
library(tidyverse)

In [ ]:
# logistic regression on the cpsdecMaine data: 
# dependent variable: 
# food_security (HRFS12MD=4 Low food security),
# independent variables:
# sex, Race (tried “white” as binary variable), 
# Country of born (us_born as binary variable), 
# employment_status (employed and unemployed), 
# poverty level,
# and disability 

In [ ]:
# logistic regression on the cpsdecMaine 2020-2022 data, and combined data for newengland: 
read_and_process <- function(file_path) {
  print(paste("Processing:", file_path))
  # Read the dataset
  data <- read_csv(file_path, show_col_types = FALSE)
  
  # Convert column names to uppercase immediately after reading the dataset
  names(data) <- toupper(names(data))
  
  # Ensure HRHHID and HRHHID2 are character types
  if("HRHHID" %in% names(data)) {
    data$HRHHID <- as.character(data$HRHHID)
  }
  if("HRHHID2" %in% names(data)) {
    data$HRHHID2 <- as.character(data$HRHHID2)
  }
  
  # Now perform data transformations
  data <- data %>%
    mutate(
      # Create binary variables while excluding unwanted values
      DISABILITY = if_else(PEMLR == 6, 1, 0),
      FOOD_INSECURITY = if_else(HRFS12MD == 4, 1, 0),
      SEX = if_else(PESEX == 1, 1, 0),
      US_BORN = if_else(PENATVTY == 57, 1,0),
      EMPLOYMENT_STATUS = if_else(PREXPLF == 1, 1, 0),
      POVERTY = if_else(HRPOOR == 1, 1, 0),
      OVER_65 = if_else(PRTAGE > 65, 1,0),
      # Handling RACE separately due to multiple conditions
      RACE = case_when(
        PTDTRACE == 1 ~ "White",
        PTDTRACE == 2 ~ "Black",
        PTDTRACE == 3 ~ "AmericanIndian",
        PTDTRACE == 4 ~ "Asian",
        TRUE ~ "Other"
      )
    ) %>%
    mutate(RACE = factor(RACE)) %>%
    mutate(RACE = relevel(RACE, ref = "White"))
  
  
  # Extract year from file path
  year <- as.integer(str_extract(file_path, "\\d{4}"))
  data$YEAR <- year
  
  # Adjust for 4 implied decimal places in HWHHWGT
  data$HWHHWGT <- data$HWHHWGT / 10000
  
  # Filter data for PEMLR == 6 and then count -1 values for specified variables
  data %>%
    dplyr::filter(PEMLR == 6) %>%
    summarise(
      Count_FOOD_INSECURITY_minus1 = sum(FOOD_INSECURITY == -1, na.rm = TRUE),
      Count_SEX_minus1 = sum(SEX == -1, na.rm = TRUE),
      Count_US_BORN_minus1 = sum(US_BORN == -1, na.rm = TRUE),
      Count_EMPLOYMENT_STATUS_minus1 = sum(EMPLOYMENT_STATUS == -1, na.rm = TRUE),
      #Count_POVERTY_minus1 = sum(POVERTY == -1, na.rm = TRUE),
      Count_OVER_65_minus1 = sum(OVER_65 == -1, na.rm = TRUE)
    ) -> negative_ones_analysis
  
  print("Count of -1 values for PEMLR == 6 in specified variables:")
  print(negative_ones_analysis)
  
  # Save the processed dataset as a CSV file
  processed_file_path <- str_replace(file_path, ".csv", paste0("_PROCESSED_", year, ".csv"))
  write_csv(data, processed_file_path)
  
  return(data)
}

In [ ]:
new_england_file_paths <- c(
  "2022" = "/Users/xinyanliu/Desktop/NEU/Apriqot/CPS/2022/cpsdec22_NewEngland.csv",
  "2021" = "/Users/xinyanliu/Desktop/NEU/Apriqot/CPS/2021/cpsdec21_NewEngland.csv",
  "2020" = "/Users/xinyanliu/Desktop/NEU/Apriqot/CPS/2020/cpsdec20_NewEngland.csv",
  "2017" = "/Users/xinyanliu/Desktop/NEU/Apriqot/CPS/2017/cpsdec17_NewEngland.csv",
  "2016" = "/Users/xinyanliu/Desktop/NEU/Apriqot/CPS/2016/cpsdec16_NewEngland.csv",
  "2015" = "/Users/xinyanliu/Desktop/NEU/Apriqot/CPS/2015/cpsdec15_NewEngland.csv",
  "2014" = "/Users/xinyanliu/Desktop/NEU/Apriqot/CPS/2014/cpsdec14_NewEngland.csv",
  "2013" = "/Users/xinyanliu/Desktop/NEU/Apriqot/CPS/2013/cpsdec13_NewEngland.csv",
  "2012" = "/Users/xinyanliu/Desktop/NEU/Apriqot/CPS/2012/cpsdec12_NewEngland.csv",
  "2011" = "/Users/xinyanliu/Desktop/NEU/Apriqot/CPS/2011/cpsdec11_NewEngland.csv",
  "2010" = "/Users/xinyanliu/Desktop/NEU/Apriqot/CPS/2010/cpsdec10_NewEngland.csv"
)

In [ ]:
# Process each file separately; this will save each processed file as a new CSV
processed_data_list <- lapply(new_england_file_paths, read_and_process)

# Combine all processed datasets for analysis
all_data_processed <- bind_rows(processed_data_list)
all_data_processed <- all_data_processed[!is.na(all_data_processed$HWHHWGT), ]

In [ ]:
# Assuming no stratification and treating each observation as its own PSU
cps_design <- svydesign(ids = ~1, weights = ~HWHHWGT, data = all_data_processed)

model_svy <- svyglm(FOOD_INSECURITY ~ SEX + EMPLOYMENT_STATUS + US_BORN + RACE + DISABILITY + OVER_65+POVERTY, 
                    design = cps_design, 
                    family = binomial(link = "logit"))
summary(model_svy)

In [ ]:
# Create a summary table for the race distribution
race_distribution <- all_data_processed %>%
  count(RACE) %>%
  mutate(percentage = n / sum(n) * 100)

In [ ]:
# pie chart for the race distribution
cps_pie_chart <- ggplot(race_distribution, aes(x = "", y = percentage, fill = RACE)) +
  geom_bar(stat = "identity", width = 1) +
  coord_polar("y", start = 0) +
  theme_void() +
  theme(legend.title = element_blank()) +
  scale_fill_brewer(palette = "Set3") +
  labs(fill = "Race", title = "Percentage Distribution of Race")

# Print the pie chart
print(cps_pie_chart)

# Save the pie chart
ggsave("CPS_newengland_race.png", plot = cps_pie_chart, width = 10, height = 8)

In [ ]:
# Apply model fit coefficent to ACS dataset
process_new_dataset <- function(file_path) {
  # Read the dataset
  new_data <- read_csv(file_path, show_col_types = FALSE)
  
  # Create binary variable for age
  new_data <- new_data %>%
    mutate(
      # FOOD_INSECURITY = NA_integer_, # Placeholder, as predictions will replace this
      SEX = if_else(SEX == 1, 1, 0),
      POVERTY = if_else(POVPIP <= 185, 1, 0),
      OVER_65 = if_else(AGEP > 65, 1, 0),
      # Create binary employment status variable
      EMPLOYMENT_STATUS = if_else(ESR %in% c("1", "2", "4", "5","6"), 1, 0),
      # Convert DIS to binary variable
      DISABILITY = if_else(DIS == 1, 1, 0),
      US_BORN = if_else(NATIVITY == 1, 1, 0),
      # Recode race code of the householder
      WHITE = if_else(RAC1P == 1, 1, 0),
      BLACK = if_else(RAC1P == 2, 1, 0),
      AMERICANINDIAN = if_else(RAC1P == 3, 1, 0),
      ASIAN = if_else(RAC1P == 6, 1, 0),
      RACE = case_when(
        RAC1P == 1 ~ "White",
        RAC1P == 2 ~ "Black",
        RAC1P == 3 ~ "AmericanIndian",
        RAC1P == 6 ~ "Asian",
        TRUE ~ "Other" # Handle other cases or include additional race categories if needed
      )
    ) %>%
    mutate(RACE = factor(RACE))%>%
    mutate(RACE = relevel(RACE, ref = "White")) %>%
    select(SEX, EMPLOYMENT_STATUS,OVER_65, US_BORN, POVERTY, DISABILITY,RACE)
    # Apply drop_na
    new_data <- drop_na(new_data)

  # Summary before na.omit
  # print(summary(new_data))
  
  # Apply na.omit
  # new_data <- na.omit(new_data)
  
  # Return the processed new data
  return(new_data)
}

In [ ]:
# Read and process the new dataset
new_dataset_file_path <- "/Users/xinyanliu/Desktop/NEU/Apriqot/ACS/new_england_people_18_22.csv"
new_data_processed <- process_new_dataset(new_dataset_file_path)

summary(new_data_processed)


In [ ]:
# Apply the model to the new dataset to get predictions for FOOD_INSECURITY
predictions <- predict(model_svy, newdata = new_data_processed, type = "response")


# Add predictions back to the new data frame if you want to analyze them together
new_data_processed$FOOD_INSECURITY_PREDICTIONS <- predictions

# Output the new dataset with predictions
write_csv(new_data_processed, "/Users/xinyanliu/Desktop/NEU/Apriqot/ACS/new_england_people_18_22_prediction_add_poverty.csv")

# View summary of predictions
summary(predictions)

In [ ]:
# Convert RACE to numeric for plotting (this is just for visualization and may not be ideal for interpretation)
new_data_processed$RACE_numeric <- as.numeric(new_data_processed$RACE)

# Create a scatter plot with jitter for binary variables and use facets for RACE
# Adjust the previous plot code with a smaller size for scatter points
# Adjust the previous plot code to reflect the change to percentages
scatter_plot<- ggplot(new_data_processed, aes(y = FOOD_INSECURITY_PREDICTIONS*100)) +
  geom_jitter(aes(x = as.numeric(SEX) * 0.5, color = "SEX"), width = 0.05, alpha = 0.5, size = 0.5) +
  geom_jitter(aes(x = as.numeric(EMPLOYMENT_STATUS) * 0.5 + 2, color = "EMPLOYMENT_STATUS"), width = 0.05, alpha = 0.5, size = 0.5) +
  geom_jitter(aes(x = as.numeric(DISABILITY) * 0.5 + 4, color = "DISABILITY"), width = 0.05, alpha = 0.5, size = 0.5) +
  geom_jitter(aes(x = as.numeric(OVER_65) * 0.5 + 6, color = "OVER_65"), width = 0.05, alpha = 0.5, size = 0.5) +
  geom_jitter(aes(x = as.numeric(US_BORN) * 0.5 + 8, color = "US_BORN"), width = 0.05, alpha = 0.5, size = 0.5) +
  geom_jitter(aes(x = as.numeric(POVERTY) * 0.5 + 10, color = "POVERTY"), width = 0.05, alpha = 0.5, size = 0.5) +
  scale_x_continuous(breaks = c(0.125, 0.5, 2, 2.5, 4, 4.5, 6, 6.5, 8, 8.5,10,10.5),
                     labels = c("Female", "Male","Unemployed","Employed","No Disability","Disability","Below 65","Over 65","Non-US Born","US Born","Above-Poverty","Below-Poverty"),
                     #labels = c("0", "1", "0", "1", "0", "1", "0", "1", "0", "1","0", "1"),
                     limits = c(0, 11)) +
  scale_color_manual(values = c("SEX" = "blue", "EMPLOYMENT_STATUS" = "green", "DISABILITY" = "red", "OVER_65" = "orange", "US_BORN" = "brown","POVERTY"="black")) +
  labs(color = "Variables", x = "Independent Variables", y = "Predicted Probability of Food Insecurity (%)") +
  theme_minimal() +
  theme(legend.position = "bottom") +
  facet_wrap(~RACE, ncol = 1, labeller = labeller(RACE = c(`1` = "White", `2` = "Black", `3` = "AmericanIndian", `4` = "Asian", `5` = "Other")))

# Print the plot
print(scatter_plot)

# Save the plot if needed
ggsave("acs_new_england_food_security_prediction.png", plot = scatter_plot, width = 20, height = 8)
